# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /Users/tom/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /Users/tom/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/tom/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_table', engine)
X = df.message
y = df.drop(['id','message','original','genre'], axis=1).fillna(0)

In [4]:
X.shape, y.shape

((26345,), (26345, 36))

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower() ) 
    
    # splite text into words
    words = word_tokenize(text)
    
    # remove stop words 
    words = [w for w in words if w not in stopwords.words("english")]
    
    # lemmatization word
    words = [WordNetLemmatizer().lemmatize(w).strip() for w in words]
   
    #stemming word
    words = [PorterStemmer().stem(w) for w in words]  
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train, y_train)

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tom/opt/anaconda3/lib/python3

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [9]:
for i, col in enumerate(y_test):
    print('Categories: {}'.format(col))
    print(classification_report(y_test[col], y_pred[:, i]))
print('Accuracy: {}'.format((y_test.values == y_pred).mean()))

[0. 1. 2.]
Categories: related
              precision    recall  f1-score   support

         0.0       0.34      0.16      0.21      1540
         1.0       0.77      0.91      0.83      5012
         2.0       0.29      0.06      0.10        35

    accuracy                           0.73      6587
   macro avg       0.47      0.37      0.38      6587
weighted avg       0.67      0.73      0.69      6587

Categories: request
              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90      5476
         1.0       0.39      0.08      0.14      1111

    accuracy                           0.82      6587
   macro avg       0.62      0.53      0.52      6587
weighted avg       0.76      0.82      0.77      6587

Categories: offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6545
         1.0       0.00      0.00      0.00        42

    accuracy                           0.99      6587
   ma

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6503
         1.0       0.00      0.00      0.00        84

    accuracy                           0.99      6587
   macro avg       0.49      0.50      0.50      6587
weighted avg       0.97      0.99      0.98      6587

Categories: other_infrastructure
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      6308
         1.0       0.07      0.00      0.01       279

    accuracy                           0.96      6587
   macro avg       0.51      0.50      0.49      6587
weighted avg       0.92      0.96      0.94      6587

Categories: weather_related
              precision    recall  f1-score   support

         0.0       0.74      0.94      0.83      4732
         1.0       0.50      0.16      0.24      1855

    accuracy                           0.72      6587
   macro avg       0.62      0.55      0.54      6587
weighted avg 

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7fbf01b974d0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_feature

### 6. Improve your model
Use grid search to find better parameters. 

In [33]:
cv_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

parameters = {  
      'clf__estimator__n_estimators': [20, 50], 
      'clf__estimator__min_samples_split': [4, 6]
} 

cv = GridSearchCV(cv_pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

y_pred = cv.predict(X_test)

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
for i, col in enumerate(y_test):
    print('Categories: {}'.format(col))
    print(classification_report(y_test[col], y_pred[:, i]))
print('Accuracy: {}'.format((y_test.values == y_pred).mean()))

[0. 1. 2.]
Categories: related
              precision    recall  f1-score   support

         0.0       0.35      0.06      0.10      1540
         1.0       0.77      0.97      0.85      5012
         2.0       0.33      0.03      0.05        35

    accuracy                           0.75      6587
   macro avg       0.48      0.35      0.34      6587
weighted avg       0.67      0.75      0.67      6587

Categories: request
              precision    recall  f1-score   support

         0.0       0.84      0.98      0.91      5476
         1.0       0.46      0.07      0.12      1111

    accuracy                           0.83      6587
   macro avg       0.65      0.53      0.51      6587
weighted avg       0.77      0.83      0.77      6587

Categories: offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6545
         1.0       0.00      0.00      0.00        42

    accuracy                           0.99      6587
   ma

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      6308
         1.0       0.17      0.01      0.01       279

    accuracy                           0.96      6587
   macro avg       0.56      0.50      0.50      6587
weighted avg       0.92      0.96      0.94      6587

Categories: weather_related
              precision    recall  f1-score   support

         0.0       0.75      0.95      0.84      4732
         1.0       0.60      0.17      0.27      1855

    accuracy                           0.73      6587
   macro avg       0.67      0.56      0.55      6587
weighted avg       0.70      0.73      0.68      6587

Categories: floods
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      6057
         1.0       0.07      0.00      0.00       530

    accuracy                           0.92      6587
   macro avg       0.49      0.50      0.48      6587
weighted avg       0.85    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [30]:
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier

def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('svd', TruncatedSVD()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MLPClassifier()))
    ])
    parameters = {
     'clf__estimator__early_stopping': [False, True],
     'clf__estimator__hidden_layer_sizes': [100, 200],
     'clf__estimator__learning_rate_init': [0.001, 0.01]
    }
    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [31]:
mlp_model = build_model()
mlp_model.fit(X_train, y_train)
y_pred = mlp_model.predict(X_test)

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [32]:
for i, col in enumerate(y_test):
    print('Categories: {}'.format(col))
    print(classification_report(y_test[col], y_pred[:, i]))
print('Accuracy: {}'.format((y_test.values == y_pred).mean()))

[0. 1.]
Categories: related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1540
         1.0       0.76      1.00      0.86      5012
         2.0       0.00      0.00      0.00        35

    accuracy                           0.76      6587
   macro avg       0.25      0.33      0.29      6587
weighted avg       0.58      0.76      0.66      6587

Categories: request
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      5476
         1.0       0.00      0.00      0.00      1111

    accuracy                           0.83      6587
   macro avg       0.42      0.50      0.45      6587
weighted avg       0.69      0.83      0.75      6587

Categories: offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6545
         1.0       0.00      0.00      0.00        42

    accuracy                           0.99      6587
   macro

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6446
         1.0       0.00      0.00      0.00       141

    accuracy                           0.98      6587
   macro avg       0.49      0.50      0.49      6587
weighted avg       0.96      0.98      0.97      6587

Categories: missing_people
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6511
         1.0       0.00      0.00      0.00        76

    accuracy                           0.99      6587
   macro avg       0.49      0.50      0.50      6587
weighted avg       0.98      0.99      0.98      6587

Categories: refugees
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6370
         1.0       0.00      0.00      0.00       217

    accuracy                           0.97      6587
   macro avg       0.48      0.50      0.49      6587
weighted avg       0.94   

### 9. Export your model as a pickle file

In [36]:
import pickle

mlp_file_name = "pickle_mlp_model.pkl"  

with open(mlp_file_name, 'wb') as file:  
    pickle.dump(mlp_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import libraries
import sys  # cmd input

import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier

def load_data(db_file_path):
    engine = create_engine('sqlite:///{}'.format(db_file_path))
    df = pd.read_sql_table('messages_table', engine)
    X = df.message
    y = df.drop(['id','message','original','genre'], axis=1).fillna(0)
    return X, y

def tokenize(text):
    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower() ) 
    # splite text into words
    words = word_tokenize(text)
    # remove stop words 
    words = [w for w in words if w not in stopwords.words("english")]
    # lemmatization word
    words = [WordNetLemmatizer().lemmatize(w).strip() for w in words]
    #stemming word
    words = [PorterStemmer().stem(w) for w in words]  
    return words

def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('svd', TruncatedSVD()),
        ('tfidf', TfidfTransformer()),
        ('clf', MLPClassifier())
    ])
    parameters = {
    'clf__early_stopping': (False, True),
    'clf__learning_rate': ('constant', 'invscaling', 'adaptive'),
    }
    model_pipeline = GridSearchCV(pipeline, param_grid=parameters)
    return model_pipeline


def train(X, y, model):
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    # fit model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model


def export_model(model):
    # Export model as a pickle file
    mlp_file_name = "pickle_mlp_model.pkl"  
    with open(mlp_file_name, 'wb') as file:  
        pickle.dump(model, file)


def run_pipeline(data_file):
    X, y = load_data(data_file)  # run ETL pipeline
    model = build_model()  # build model pipeline
    model = train(X, y, model)  # train model pipeline
    export_model(model)  # save model


if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline